# Script for transforming raw mobility data to edge table and trips per day and district

## Specify identifier for output files

In [ ]:
identifier = 'teralytics_2020_'

## Import libraries

In [ ]:
import pandas as pd
import numpy as np

## Read in raw data from file

Read in file into dataframe:

In [ ]:
df_raw = pd.DataFrame()

for chunk in pd.read_csv('raw_mobility_data/FILENAME_HERE.csv', chunksize=10**6):
    df_raw = df_raw.append(chunk)

df_raw.reset_index(inplace=True, drop=True)

Drop columns we don't need:

In [ ]:
df_raw.drop(columns=['Mot','DistanceInKm'], inplace=True)

Specify columns names with origin and destination districts, dates and mobility counts:

In [ ]:
origin_col_name = 'StartId'
destination_col_name = 'EndId'
date_col_name = 'DayLocal'
count_col_name = 'Count'

## OSM ID to AGS5

In [ ]:
o2a_df = pd.read_csv('https://raw.githubusercontent.com/J-Driesen/PD_goes_viral/master/OSM_2_AGS_v2.csv')

In [ ]:
osm_2_ags_dict = dict(zip(o2a_df.OSM_ID,o2a_df.AGS_5))

In [ ]:
df_raw['orig_ags5'] = df_raw[origin_col_name].map(osm_2_ags_dict)
df_raw['dest_ags5'] = df_raw[destination_col_name].map(osm_2_ags_dict)


In [ ]:
df_raw.drop(columns=[origin_col_name,destination_col_name], inplace=True)

## Create edge table

In [ ]:
edge_table = df_raw.groupby([date_col_name,'orig_ags5','dest_ags5'])[count_col_name].sum().reset_index()

Save edge table to CSV-file using the specifier defined above:

In [ ]:
file_name = identifier + 'edge_table.csv'
edge_table.to_csv(file_name, index=False)

In case there's a problem later on, reading in this CSV will serve as a checkpoint:

In [ ]:
# edge_table = pd.read_csv(file_name)

## Create table for trips per district and day

Create Boolean Series to subset data into within- and between-district mobility:

In [ ]:
is_internal = edge_table.orig_ags5 == edge_table.dest_ags5

Create multi-indexed Series for outgoing, incoming and internal mobility per district and day:

In [ ]:
outgoing_mobility = edge_table[~is_internal].groupby([date_col_name, 'orig_ags5'])[count_col_name].sum().reset_index()
incoming_mobility = edge_table[~is_internal].groupby([date_col_name, 'dest_ags5'])[count_col_name].sum().reset_index()
internal_mobility = edge_table[ is_internal].groupby([date_col_name, 'orig_ags5'])[count_col_name].sum().reset_index()

Merge all three into one dataframe, clean that dataframe up, and add total mobility:

In [ ]:
external_mobility = pd.merge(outgoing_mobility, incoming_mobility, 
                   left_on = [date_col_name, 'orig_ags5'], right_on = [date_col_name, 'dest_ags5'],
                   suffixes = ('_outgoing','_incoming'))

mobility_per_district_day = pd.merge(external_mobility, internal_mobility,
                                    on = [date_col_name, 'orig_ags5'])

#rename columns:
mobility_per_district_day.rename(columns = {count_col_name : count_col_name + '_internal',
                                           'orig_ags5' : 'districtId'}, 
                                 inplace=True)

#drop redundant dest_ags5 column:
mobility_per_district_day.drop(columns = 'dest_ags5', inplace=True)

#add total_mobility as sum:
count_cols = [count_col_name + suffix for suffix in ['_outgoing','_incoming','_internal']]
mobility_per_district_day['total_mobility'] = mobility_per_district_day[count_cols].sum(axis=1)

Save mobility per district and day to CSV, again using the identifier:

In [ ]:
file_name = identifier + 'mobility_per_district_and_day.csv'
mobility_per_district_day.to_csv(file_name, index=False)